In [ ]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import xlsxwriter

import pyodbc
with open('login.txt') as key:
    login=key.read().strip()

cnxn = pyodbc.connect(login)

sql_query="""
SELECT wonum, convert(date, dbo.workorder.schedstart) 'Start Date', concat(person.firstname,' ',person.lastname) as 'Tech', 
    convert (numeric, pmnum) pmnum
FROM workorder

FULL JOIN person on person.utln = workorder.[lead]

WHERE workorder.status not in ('CAN','CLOSE','COMP','WPLAN','WCLIENT','NOTPERF') 
and workorder.siteid = 'BOSTON'
and dbo.workorder.istask = 0
and dbo.workorder.historyflag = 0
and dbo.workorder.[lead] is not null
order by dbo.workorder.[lead], 'Start Date'
"""

data = pd.read_sql(sql_query,cnxn, parse_dates='Start Date')
pd.options.display.float_format = '{:.0f}'.format


#create a dataframe with the tech names as the index

techs = data['Tech'].unique()
cols=['Total WO', 'non-PMs', 'PMs', '0-14 days', '15-30 days', '31-60 days', '60+ days', 'Future start date', 'No start date']
report = pd.DataFrame(index=techs,columns=cols)
report.index.name = 'Lead'
report['Total WO'] = data['Tech'].value_counts()

#techs total number of non-PM WO
report['non-PMs'] = data['Tech'][pd.isnull(data['pmnum'])].value_counts()

#number of PMs
report['PMs'] = data['Tech'][pd.notnull(data['pmnum'])].value_counts()

#aging
today = dt.now()

#function to find count of WOs between days. 
def between_dates(start,end):
    start_date=today-timedelta(days=end)
    end_date = (today-timedelta(days=start))
    return data['Tech'][data['Start Date'].between(start_date,end_date, inclusive=True)].value_counts()

report['0-14 days'] = data['Tech'][data['Start Date']>(today-timedelta(days=14))].value_counts()
report['15-30 days'] = between_dates(15,30)
report['31-60 days'] = between_dates(31,60)
report['60+ days'] = between_dates(61,1000)
report['Future start date'] = data['Tech'][data['Start Date']>today].value_counts()
report['No start date'] = data['Tech'][pd.isnull(data['Start Date'])].value_counts()

report.fillna("", inplace=True)
display(report)

In [ ]:
#export to excel
def get_answers(question,choices):
  choice = ""
  while choice.lower() not in choices:
        
        choice = input(question + " Choose [%s]:" % ", ".join(choices))
        choice = choice.lower()
  return choice

file_name = today.strftime('%Y.%m.%d')+  " Aging WO Report.xlsx"
print (file_name)
export_file=get_answers("Export file?",['y', 'n'])
if export_file == "y":
#     wos_df.to_excel(file_name)
    writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

    # Convert the dataframe to an XlsxWriter Excel object.
    report.to_excel(writer, sheet_name='Aging')

    # Get the xlsxwriter workbook and worksheet objects.
    workbook = writer.book
    worksheet = writer.sheets['Aging']
    
#     header = workbook.add_format({'bold': True, 'bottom': 2, 'bg_color': '#F9DA04'})
    
    cell_format = workbook.add_format()
    cell_format2 = workbook.add_format()
    cell_format.set_text_wrap()
    cell_format.set_align('top')
    cell_format.set_align('center')
    cell_format.set_bottom()
    
    cell_format2.set_align('top')
    cell_format2.set_align('center')
    cell_format2.set_bottom()
    cell_format2.set_right()
    
    # Set the column width and format.
    col_a_width = len(max(list(techs)))+2
    worksheet.set_column('A:A',col_a_width,cell_format2)
    worksheet.set_column('B:B', 8,cell_format2)
    worksheet.set_column('C:C', 9,cell_format)
    worksheet.set_column('D:D',7,cell_format2)
    worksheet.set_column('E:E',10,cell_format)
    worksheet.set_column('F:F',10,cell_format)
    worksheet.set_column('G:G',10, cell_format)
    worksheet.set_column('H:H',10,cell_format)
    worksheet.set_column('I:I',14,cell_format)
    worksheet.set_column('J:J',11,cell_format2)

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [ ]:
#Look for duplicate PM work orders

#make a list of the PM numbers that occur more than once in the list of work orders
pms=data['pmnum'].value_counts()[data['pmnum'].value_counts()>1].index
pms = list(pms)

# create a list of work orders with the PM numbers listed more than once
data[data['pmnum'].isin(pms)].sort_values(by='pmnum')

wo_to_cancel= pd.DataFrame(columns = ['wonum','Start Date','Tech','pmnum'])

for pm in pms:
    list_of_pms = []
    list_of_pms = data[data['pmnum']==pm]
    list_of_pms.sort_values(by='Start Date')
    to_cancel = list_of_pms.iloc[:-1]
    wo_to_cancel = wo_to_cancel.append (to_cancel)
    print (list_of_pms)
    print('Delete:\n',to_cancel,'\n')
    
#try to re-write this using pd.df.duplicated(look at the keep parameter to keep only the most recent)
display(wo_to_cancel)

In [ ]:
#data is list of work orders

duplicated_pms = data[data['pmnum'].duplicated(keep=False)].dropna()
duplicated_pms.sort_values(by='pmnum')
duplicated_pms